In [1]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String

In [2]:
def transform(img):
    
    global bearings
    global tags
    
# Once a color image has been read, it passes through the 
# transform function. The transform function has two parts:
# pre-processing also known as pp which filters the image based
# of certain values in the hue, saturation, and value (hsv) 
# colorspace and the dbscan which clusters the processed image
# and classifies the clusters as either a buoy, a green buoy,
# a red buoy, or noise. No changes are needed.
    
    start_time = time.time()
    bridge = cv_bridge.CvBridge()
    bearings = []
    tags = []
    width = img.height
    height = img.width
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  

    image = pp(img,width,height)
    img2= dbscan(image, img)
    return img2
def pp(image, width, height):
    
# The first function in transform is pp or preprocessing. The 
# first line truncates the lower half of the image. The reason
# this is because the plantoons from the wamv are always 
# blocking the camera vision and become noisy data. The
# function changes the image to the HSV colorspace and 
# filters out the image. No changes are needed. It returns 
# the filtered image in the BGR colorspace.The input is the
# color image.

    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)

    return(image)
def dbscan(image, original):
    
# The second function in transform is dbscan. The inputs are 
# the filtered colored image and the original color image
# read from the imread statement. The output is the original
# image.This is the end of this classification algorithm.

###########################################################

# Part 1: Thresholding the image
##########################################################
# The reason we threshold the image before applying DBSCAN to 
# the points is because DBSCAN is a computational demanding 
# function. This part takes the filtered image, makes a 
# copy of it, and converts it into a grayscale image. The gray-
# scale image is thresholded at a value of 10. 0 is black and
# 255 is white. This takes the ~400,000 matrix points and 
# leaves only ~10,000 nonzero matrix points. 

    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

# After thresholding the image, we pull out those nonzero
# points and assign it to the nonzero array variable. This
# array has dimensions of n rows and 2 columns. The first 
# column is the y coordinate and the second column is the x
# coordinate. These two arrays would be combined by 
# np.column_stack which merges the two arrays at their columns.

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
#########################################################

# Creating the BDSCAN object and Sorting Array with Labels
#########################################################    
# After thresholding the filtered image and creating an
# array of nonzero coordinates, this algorithm applies a
# clustering algorithm that filters the image based off the
# array X. If X is empty, this function ends and output is 
# the original input. The first line creates the DBSCAN
# object with an epsilon of 3 and minimum samples of 20. 
# Eps refers to the size of the neighborhood and the 
# min_samples refers to the minimum number of samples required
# for that neighborhood to be considered a cluster. The user
# is free to change the two values for eps and min_samples. The
# .fit(X) applies the matrix points to the DBSCAN object.
# One of the parameters of the db scan object are the labels.
# This is invoked by the line labels = db.labels. This creates
# an array of labels that assigns a number to the image index.
# n clusters have labels that start at 0 and end with n-1.For 
# example, if an image has four clusters, the labels would
# include 0,1,2, and 3. -1 is dedicated for noise. 

    if (len(X) > 0):
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_

# The array variable combines the x and y coordinates and the 
# labels in a single array aligned by column. The array is
# sorted with the sort variable with the values in the third
# column or the labels. For example, an image with four
# clusters and noise would be labeled in order from (-1,0,1,2,3).
# The while loop iterates from the beginning of the sorted loop
# and deletes every instance of -1 or until the list is empty.
# This while loop is needed because the noise data is
# irrelevant to the cluster algorithm and the user does not 
# know how much noise there is in the image. The image could 
# have no noise or it could all be noise. The output of this
# section is an array with coordinates sorted together in 
# clusters.

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]
#########################################################

# Iterating through and classifying each cluster
#########################################################
# After iterating through sort, the program is either left with
# an empty array or an array of coordinates sorted and grouped
# by each cluster. The user needs the number of clusters. The 
# next three lines in the big if block does this. unique_labels
# sorts the set of labels and removes -1 if present. The next 
# two lines creates a copy of the sort array and removes the 
# labels column from the array.

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

# The program assigns three variables. The first one i iterates
# through sort and compare its label value to x for each 
# iteration until the very end of the sort array. The second
# x refers to the cluster that the program is currently class-
# ifying. It starts with 0 and continues classifying until it 
# has classified all clusters. The third variable is l and this
# array becomes all the points within a cluster. Every time the 
# variable i iterates through sort and is equal to the value of
# the current cluster x, the coordinates are appended to the 
# array l. Since this array is sorted, the first time the x 
# variable equals a different value- the cluster has been 
# accounted for. This array of l contains all the points within
# the cluster. This array and the filtered image gets passed
# through the color function. The result of the color function
# gets passed through the rectangle function with the array,
# filtered image, and original image. The x variable is 
# incremented for the next cluster and the l array is set equal 
# to zero. Once the while loop has iterated through all 
# clusters, the result of the rectangle function is set to the 
# original image and the original image is returned to the 
# transform function which is returned to the read function 
# which writes the image.

            i = 0
            x = 0
            l = []
            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color_hsv(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original)
                    l = []
            original = tb
    return original
def color(mat,image):
    
    ############################################################

# For the color function, the points of the cluster and the 
# filtered image are passed through. In this function, the 
# average red, green, and blue pixel values are calculated
# from the image inside of the cluster. If the average value
# fits within a certain range, the string 'red' or 'green' is 
# returned. Otherwise, the function returns the string 'none'
# The user is free to change the range of the BGR values that
# determine whether the buoy is red or green.

    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    if ((sumred/len(mat)) > 80 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        return 'red'
    if ((sumred/len(mat)) > 30 and (sumred/len(mat)) < 80 and (sumgrn/len(mat)) > 60 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        return 'green'
    else:
        return 'none'
def color_hsv(mat, image):
    
# The rectangle function's parameters are the array of coordinates
# of the cluster, the color of the cluster, the filtered image,
# and the original image. The boundingRect function applies
# a non-rotated rectangle to the image points and returns four
# values in an array, the x and y coordinates of the leftmost
# corner and the width and height. These values are converted
# to ints. The font is set to Hershey Simple for the text and
# the tag is set to zero. The tag is useful in the classifica-
# tion of the buoy. The first classification of the buoy is 
# the proportion of the width and height. If the height is
# between 1.5 times the width and 3.5 times the width, the
# cluster is considered a buoy. If the cluster is given a 
# 'red' or 'green' color, it would be classified as a 'red buoy'
# or 'green buoy' respectively. Otherwise, it would simply be
# classified as a buoy. If any of these three options occur, 
# the tag is set to 1. If the height is not in that range
# mentioned above and the color is 'none', the cluster is
# classified as noise. The last function rectangle makes a 
# rectangle over the original image. The original image is sent
# back to the DBSCAN function.


    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sumhue = 0
    sumsat = 0
    sumval = 0
    mat_add = 0
    for x in range(len(mat)):
        if(image [mat[x][1]] [mat[x][0]] [2] > 5):
            mat_add = mat_add+1
            sumval += image [mat[x][1]] [mat[x][0]] [2]
            sumsat += image [mat[x][1]] [mat[x][0]] [1]
            sumhue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sumsat/mat_add
        val = sumval/mat_add
        hue = sumhue/mat_add
        #return (hue, sat, val)
        if (sat > 5 and val > 5 and (hue < 60 or hue >= 170)):
            return 'red'
        if (sat > 5 and val > 5 and (70 < hue < 90)):
            return 'green'
        else:
            return (hue, sat, val)

    return None

In [3]:
def createLineIterator(P1, P2, img):
# """
# Produces and array that consists of the coordinates and intensities of
# each pixel in a line between two points

# Parameters:
#     -P1: a numpy array that consists of the coordinate of the first point (x,y)
#     -P2: a numpy array that consists of the coordinate of the second point (x,y)
#     -img: the image being processed

# Returns:
#     -it: a numpy array that consists of the coordinates and intensities of each
#     pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
# """
   #define local variables for readability
    #print img.shape
    image_height = img.shape[0]
    image_width = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]
    
   #difference and absolute difference between points
   #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3), dtype=np.int32)
    itbuffer.fill(np.nan)

   #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<image_width) & (colY<image_height)]

   #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]
    return itbuffer

In [4]:
# Initialization Stuff

rospy.init_node('liveproc')
listener = tf.TransformListener()
listener.waitForTransform("/odom", "/front_left_camera_link", rospy.Time(), rospy.Duration(2.0))

occ_map = None
map_resolution = 0.5
origin = None
robot_x = 0
robot_y = 0
map_width = 0
map_height = 0
camera_range = 75/map_resolution
overlay = False
display_robot_yaw = False
display_rect = True
display_map = True

np.set_printoptions(threshold=sys.maxsize)

In [5]:
def rectangle(mat,color,image,original):
    
    global bearings
    global overlay
    global occ_map
    global tags
    
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 2
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
                tags.append(tag)
            elif(color =='red'):
                tag = 3
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            if occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
        return original
    else:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 2
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            elif(color =='red'):
                tag = 3
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            if occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        return image

In [6]:
def get_mapnbot_info(trans,robot_yaw):

    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    
    map_array = occ_map.data

    map_width = occ_map.info.width
    map_height = occ_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))
    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
    
    return map_np, robot_yaw

In [7]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [8]:
def map_filter(trans, robot_yaw):
    
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    
##############################################################################
# This is for visualization purposes.
# Set to false if you don't want to display the robot_yaw
# Delete it if it's irritating to look at
    if display_robot_yaw and occ_map.header.seq % 5 == 0:
        yaw_offset.append(0.0)
        print "Robot_yaw: ", robot_yaw
        print "Buoy yaw: ", buoy_yaw
        print len(slices)
##############################################################################
    
    slices = []
    i = 0
    while (i < len(bearings)):
        buoy_yaw = robot_yaw + bearings[i]
        if ((buoy_yaw) < 0):
            buoy_yaw = buoy_yaw + 360
        point_deltax = camera_range * math.sin(math.radians(buoy_yaw))
        point_deltay = camera_range * math.cos(math.radians(buoy_yaw))
        
        point1 = np.array([round(robot_x), round(robot_y)])
        point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)])
        sliced = createLineIterator(point1.astype(int),point2.astype(int),map_np)
        slices.append(sliced)
        
        point3 = np.array([round(robot_x+1), round(robot_y)])
        point4 = np.array([round(robot_x+point_deltax+1), round(robot_y+point_deltay)])
        sliced = createLineIterator(point3.astype(int),point4.astype(int),map_np)
        slices.append(sliced)
        
        point5 = np.array([round(robot_x-1), round(robot_y)])
        point6 = np.array([round(robot_x+point_deltax-1), round(robot_y+point_deltay)])
        sliced = createLineIterator(point5.astype(int),point6.astype(int),map_np)
        slices.append(sliced)
        i+=1
    
    return slices

In [9]:
def draw_background_objects(map_img,map_np):
    
    global origin
    global robot_x
    global robot_y
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
# Color Assignment Section
    map_img[map_np == -1] = [255,255,255]    
    map_img[map_np == 0] = [255,255,255]
    map_img[map_np == 100] = [0,0,0]
    origin_color = [0,0,255]
    robot_color = [0,0,255]
# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color
# Robot (Drawn as a square)
    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color
    
    return map_img

In [10]:
def draw_ray(map_img,slices):
    
    global display_robot_yaw
    global bearings
    global tags
    
    slices_size = len(slices)
    
    if not(display_robot_yaw):
        for i in range(int(slices_size)):
            try:
                found = False
                arr = slices[i]
                for ray in arr:
                    #print slice
                    if not found and (ray[2] == -1 or ray[2] == 0):
                        map_img[ray[1]][ray[0]] = [255,0,0]
                    elif ray[2] == 100:
                        if tags[i//3] == 2:
                            map_img[ray[1]][ray[0]]=[0,255,0]
                            found = True
                        if tags[i//3] == 3:
                            map_img[ray[1]][ray[0]]=[0,0,255]
                            found = True
            except Exception as ex:
                print ex
                print slices[i]
                print
    else:
        for arr in slices[-1]:
            map_img[arr[1]][arr[0]] = [0,255,0]
    return map_img

In [11]:
def get_map(slices):
    
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))

    map_img = np.zeros((map_height, map_width, 3))
    
    map_img = draw_background_objects(map_img,map_np)
    
    map_img = draw_ray(map_img,slices)
    
    return map_img

In [12]:
def disp_map(map_img):

    map_img_i = np.zeros((map_width, map_height, 3))
    
    x=0
    y=0
    
    for x in range(map_width):
        for y in range(map_height):
            map_img_i[x][y] = map_img[y][x]

    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img_i)
    cv.waitKey(1)

In [13]:
def callback(rect):
    
    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags

    origin = occ_map.info.origin.position
    rect = transform(rect)
        
    if not rospy.is_shutdown():       
        try:
            now = rospy.Time.now()
            listener.waitForTransform("/odom", "/base_link", now, rospy.Duration(2.0))
            (trans,rot) = listener.lookupTransform("/odom", "/base_link", now)
            euler = tf.transformations.euler_from_quaternion(rot)
            pixel_lines = map_filter(trans, math.degrees(euler[2]))
            map_image = get_map(pixel_lines)
            if display_rect == True:
                disp_rect(rect)
            if display_map == True:
                disp_map(map_image)
        except tf.LookupException as l:
            print l
        except tf.ConnectivityException as c:
            print c
        except tf.ExtrapolationException as e:
            print e
        except Exception as ex:
            print ex

In [14]:
def map_callback(p_map):
    
    global occ_map
    
    occ_map = p_map    

In [ ]:
if __name__ == '__main__':
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()   

[35.15625]
[3]
[35.15625, -4.21875]
[3, 3]
[35.14482879638672]
[3]
[35.14482879638672, -4.21875]
[3, 3]
[35.14482879638672, -4.21875, 0.4375]
[3, 3, 2]
[-4.21875]
[3]
[-4.21875, 34.99974060058594]
[3, 3]
[-4.21875, 34.99974060058594, 0.4375]
[3, 3, 2]
[-4.15625]
[3]
[-4.15625, 35.139244079589844]
[3, 3]
[-4.15625, 35.139244079589844, 0.53125]
[3, 3, 2]
[35.3905029296875]
[3]
[35.3905029296875, -4.071693420410156]
[3, 3]
[35.3905029296875, -4.071693420410156, 0.71875]
[3, 3, 2]
[-4.03125]
[3]
[-4.03125, 35.273712158203125]
[3, 3]
[-4.03125]
[3]
[-4.03125, 35.35050964355469]
[3, 3]
[-4.03125, 35.35050964355469, 0.75]
[3, 3, 2]
[35.412132263183594]
[3]
[35.412132263183594, -4.120429992675781]
[3, 3]
[35.412132263183594, -4.120429992675781, 0.75]
[3, 3, 2]
[-4.0]
[3]
[-4.0, 35.26983642578125]
[3, 3]
[-4.0, 35.26983642578125, 0.78125]
[3, 3, 2]
[-4.0]
[3]
[-4.0, 35.39445495605469]
[3, 3]
[-4.0, 35.39445495605469, 0.8125]
[3, 3, 2]
[35.40625]
[3]
[35.40625, -3.96875]
[3, 3]
[35.40625, -3.968

[-17.53125]
[3]
[-17.53125, -6.40625]
[3, 2]
[-17.53125, -6.40625, -10.875]
[3, 2, 3]
[-17.5625]
[3]
[-17.5625, -6.40625]
[3, 2]
[-17.5625, -6.40625, -10.875]
[3, 2, 3]
[-17.5625, -6.40625, -10.875, 12.564682006835938]
[3, 2, 3, 2]
[-17.59375]
[3]
[-17.59375, -6.40625]
[3, 2]
[-17.59375, -6.40625, -10.96875]
[3, 2, 3]
[-17.59375, -6.40625, -10.96875, 12.78125]
[3, 2, 3, 2]
[-17.58942413330078]
[3]
[-17.58942413330078, -6.4375]
[3, 2]
[-17.58942413330078, -6.4375, -11.244489669799805]
[3, 2, 3]
[-17.58942413330078, -6.4375, -11.244489669799805, 12.90625]
[3, 2, 3, 2]
[-17.625]
[3]
[-17.625, -11.327232360839844]
[3, 3]
[-17.625, -11.327232360839844, -6.4375]
[3, 3, 2]
[-17.625, -11.327232360839844, -6.4375, 12.9375]
[3, 3, 2, 2]
[-17.625]
[3]
[-17.625, -6.4375]
[3, 2]
[-17.5625]
[3]
[-17.5625, -6.4375]
[3, 2]
[-17.5625, -6.4375, -11.21114730834961]
[3, 2, 3]
[-17.561206817626953]
[3]
[-17.561206817626953, -6.4375]
[3, 2]
[-17.561206817626953, -6.4375, -10.741281509399414]
[3, 2, 3]
[-17.

[-4.59375]
[3]
[-4.59375, 6.15625]
[3, 2]
[-4.59375, 6.15625, 1.375]
[3, 2, 3]
[-4.65625]
[3]
[-4.65625, 6.21875]
[3, 2]
[-4.65625, 6.21875, 1.40625]
[3, 2, 3]
[-4.724521636962891]
[3]
[-4.724521636962891, 6.21875]
[3, 2]
[-4.724521636962891, 6.21875, 1.40625]
[3, 2, 3]
[-4.625]
[3]
[-4.625, 6.1875]
[3, 2]
[-4.625, 6.1875, 1.46875]
[3, 2, 3]
[-4.625]
[3]
[-4.625, 6.15625]
[3, 2]
[-4.625, 6.15625, 1.5625]
[3, 2, 3]
[-4.625, 6.15625, 1.5625, 28.780601501464844]
[3, 2, 3, 2]
[-4.65625]
[3]
[-4.65625, 6.21875]
[3, 2]
[-4.65625, 6.21875, 1.4913482666015625]
[3, 2, 3]
[-4.65625]
[3]
[-4.65625, 6.21875]
[3, 2]
[-4.65625, 6.21875, 1.5480270385742188]
[3, 2, 3]
[-4.65625, 6.21875, 1.5480270385742188, 29.32062530517578]
[3, 2, 3, 2]
[-4.625]
[3]
[-4.625, 6.156253814697266]
[3, 2]
[-4.625, 6.156253814697266, 1.3021659851074219]
[3, 2, 3]
[-4.625, 6.156253814697266, 1.3021659851074219, 29.324363708496094]
[3, 2, 3, 2]
[-4.65625]
[3]
[-4.65625, 6.21875]
[3, 2]
[-4.65625, 6.21875, 1.7769279479980469

[-4.46875, 6.21875]
[3, 2]
[-4.46875, 6.21875, 2.0625]
[3, 2, 3]
[-4.46875]
[3]
[-4.46875, 6.25]
[3, 2]
[-4.46875, 6.25, 1.75]
[3, 2, 3]
[-4.46875, 6.25, 1.75, 28.677886962890625]
[3, 2, 3, 2]
[-4.46875]
[3]
[-4.46875, 6.220970153808594]
[3, 2]
[-4.46875, 6.220970153808594, 1.9220848083496094]
[3, 2, 3]
[-4.46875, 6.220970153808594, 1.9220848083496094, 28.90630340576172]
[3, 2, 3, 2]
[-4.46875]
[3]
[-4.46875, 6.21875]
[3, 2]
[-4.46875, 6.21875, 1.75]
[3, 2, 3]
[-4.46875]
[3]
[-4.46875, 6.28125]
[3, 2]
[-4.46875, 6.28125, 1.78125]
[3, 2, 3]
[-4.46875, 6.28125, 1.78125, 29.25]
[3, 2, 3, 2]
[-4.521579742431641]
[3]
[-4.521579742431641, 6.3125]
[3, 2]
[-4.521579742431641, 6.3125, 1.75]
[3, 2, 3]
[-4.5034027099609375, 6.28125]
[3, 2]
[-4.5034027099609375, 6.28125, 1.9226036071777344]
[3, 2, 3]
[-4.5034027099609375, 6.28125, 1.9226036071777344, 28.71875762939453]
[3, 2, 3, 2]
[-4.480144500732422]
[3]
[-4.480144500732422, 6.25]
[3, 2]
[-4.480144500732422, 6.25, 1.75]
[3, 2, 3]
[-4.48014450073

[-4.28125]
[3]
[-4.28125, 6.34375]
[3, 2]
[-4.28125, 6.34375, 2.0625]
[3, 2, 3]
[-4.3125]
[3]
[-4.3125, 6.34375]
[3, 2]
[-4.3125, 6.34375, 2.0005569458007812]
[3, 2, 3]
[-4.3125, 6.34375, 2.0005569458007812, 28.46875]
[3, 2, 3, 2]
[-4.28125]
[3]
[-4.28125, 6.375]
[3, 2]
[-4.28125, 6.375, 2.09375]
[3, 2, 3]
[-4.28125, 6.375, 2.09375, 28.406272888183594]
[3, 2, 3, 2]
[-4.28125]
[3]
[-4.28125, 6.309486389160156]
[3, 2]
[-4.25]
[3]
[-4.25, 6.2520599365234375]
[3, 2]
[-4.25, 6.2520599365234375, 2.0625]
[3, 2, 3]
[-4.28125]
[3]
[-4.28125, 6.34375]
[3, 2]
[-4.28125, 6.34375, 2.09375]
[3, 2, 3]
[-4.3125]
[3]
[-4.3125, 6.375]
[3, 2]
[-4.3125, 6.375, 2.0625]
[3, 2, 3]
[-4.3125, 6.375, 2.0625, 28.71875]
[3, 2, 3, 2]
[-4.28125]
[3]
[-4.28125, 6.3125]
[3, 2]
[-4.28125, 6.3125, 2.0625]
[3, 2, 3]
[-4.351932525634766]
[3]
[-4.351932525634766, 6.375]
[3, 2]
[-4.351932525634766, 6.375, 2.6875]
[3, 2, 3]
[-4.25]
[3]
[-4.25, 6.3125]
[3, 2]
[-4.25, 6.3125, 2.385101318359375]
[3, 2, 3]
[-4.25, 6.3125, 2.385

[-4.125]
[3]
[-4.125, 6.40625]
[3, 2]
[-4.125, 6.40625, 2.34375]
[3, 2, 3]
[-4.125, 6.40625, 2.34375, 28.125]
[3, 2, 3, 2]
[-4.125]
[3]
[-4.125, 2.375]
[3, 3]
[-4.125, 2.375, 6.375]
[3, 3, 2]
[-4.125, 6.40625, 2.375]
[3, 2, 3]
[-4.125, 6.40625, 2.375, 28.21875]
[3, 2, 3, 2]
[-4.09375]
[3]
[-4.09375, 6.40625]
[3, 2]
[-4.09375, 6.466678619384766, 2.40625, 28.00005340576172]
[3, 2, 3, 2]
[-4.09375, 6.466678619384766, 2.40625, 28.00005340576172, -34.990657329559326]
[3, 2, 3, 2, 2]
